In [14]:
import Utils as utils
import Collaborative_Filtering_Utils as cf_utils

import pandas as pd
import numpy as np
import pickle

from surprise import SVD, SVDpp, SlopeOne, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import accuracy

import matplotlib
matplotlib.use('MacOSX')
%matplotlib inline
from matplotlib import pyplot as plt


In [ ]:
################## Load trained algorithms ##################
'''file = open('/home/Virality_Predictor/models/CF_EN_PT_norm.pkl',"rb")
algo_norm = pickle.load(file)
file.close()

file = open('/home/Virality_Predictor/models/CF_EN_PT_raw.pkl',"rb")
algo_raw = pickle.load(file)
file.close()'''


In [2]:
# Load articles
shared_articles_df = utils.load_shared_articles('shared_articles.csv', ['en', 'pt'])
article_indices = pd.Series(shared_articles_df.index)


/Users/mirayyuce/Desktop/Projects/Virality_Predictor/Utils.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['domain_name'] = df['url'].apply(split_url)


In [3]:
# Load users with raw virality values
user_interactions_df_raw = utils.load_user_interactions('users_interactions.csv', shared_articles_df, to_normalize=False)

# Loads users with normalized virality
user_interactions_df_norm = utils.load_user_interactions('users_interactions.csv', shared_articles_df, to_normalize=True)

In [4]:
print('Are there duplicate users?')
user_interactions_df_raw['personId'].nunique() != len(user_interactions_df_raw.groupby('personId'))

Are there duplicate users?


False

In [5]:
print('Number of users')
user_interactions_df_raw['personId'].nunique()

Number of users


1894

In [6]:
# Discard users with only one interaction for normalized virality
user_interactions_df_norm = utils.filter_user_interactions(user_interactions_df_norm, 1)

Number of users discarded:  275


In [7]:
# Discard users with only one interaction for raw virality
user_interactions_df_raw = utils.filter_user_interactions(user_interactions_df_raw, 1)

Number of users discarded:  275


In [8]:
# Create train and test datasets for normalized virality
norm_train_data_object, norm_test_data_object, norm_user_interactions_data_object = cf_utils.prepare_train_test_splits_cross_validate(user_interactions_df_log, is_normalized=True)
norm_built_full_train, norm_anti_test_set_from_train, norm_built_full_test_set, norm_train_to_test_set = cf_utils.prepare_train_test_sets(norm_train_data_object, norm_test_data_object)

normalized_sets = {'cv_train': norm_train_data_object,
                   'cv_test': norm_test_data_object,
                   'k_fold_set': norm_user_interactions_data_object,
                   'trainset': norm_built_full_train,
                   'testset' : norm_built_full_test_set,
                   'trainset_test_it' : norm_train_to_test_set,
                   'anti_trainset': norm_anti_test_set_from_train}


In [9]:
# Create train and test datasets for raw virality
raw_train_data_object, raw_test_data_object, raw_user_interactions_data_object = cf_utils.prepare_train_test_splits_cross_validate(user_interactions_df_raw, is_normalized=False)
raw_built_full_train, raw_anti_test_set_from_train, raw_built_full_test_set, raw_train_to_test_set = cf_utils.prepare_train_test_sets(raw_train_data_object, raw_test_data_object)

raw_sets = {'cv_train': raw_train_data_object,
            'cv_test': raw_test_data_object,
            'k_fold_set': raw_user_interactions_data_object,
            'trainset': raw_built_full_train,
            'testset' : raw_built_full_test_set,
            'trainset_test_it' : raw_train_to_test_set,
            'anti_trainset': raw_anti_test_set_from_train}

In [10]:
# Search for promising algorithm using raw data
norm_results = cf_utils.algorithm_search(normalized_sets['cv_train'])
norm_results

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity

,test_rmse,fit_time,test_time,test_mse,test_mae
Algorithm,,,,,
BaselineOnly,0.506450,0.112089,0.223660,NaN,NaN
SVDpp,0.508199,93.057983,4.404996,NaN,NaN
SVD,0.513221,7.973631,0.609095,NaN,NaN
KNNWithZScore,0.528549,0.274220,0.850521,NaN,NaN
KNNBaseline,0.530231,0.427428,1.024967,NaN,NaN
KNNWithMeans,0.530247,0.180880,0.772812,NaN,NaN
SlopeOne,0.542214,0.630629,1.341873,NaN,NaN
KNNBasic,0.563673,0.119922,0.679245,NaN,NaN
CoClustering,0.568914,2.275413,0.150055,NaN,NaN


In [11]:
raw_results = cf_utils.algorithm_search(raw_sets['cv_train'])
raw_results

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity

,test_rmse,fit_time,test_time,test_mse,test_mae
Algorithm,,,,,
BaselineOnly,4.722996e+01,0.083110,0.085659,NaN,NaN
KNNWithMeans,4.863959e+01,0.199620,1.166403,NaN,NaN
KNNBaseline,4.933695e+01,0.186518,1.058698,NaN,NaN
CoClustering,5.787061e+01,1.349890,0.174318,NaN,NaN
KNNWithZScore,6.234573e+01,0.257068,0.625194,NaN,NaN
NormalPredictor,6.309050e+01,0.053506,0.139172,NaN,NaN
KNNBasic,6.499635e+01,0.167477,0.786028,NaN,NaN
SlopeOne,9.249428e+01,0.591189,1.081786,NaN,NaN
SVD,9.999061e+06,3.266889,0.221201,NaN,NaN


In [ ]:
# Try SlopeOne

In [12]:
# Train and test raw data
algo_raw = SlopeOne()
algo_raw.fit(raw_sets['trainset'])
predictions_raw_anti_trainset = algo_raw.test(raw_sets['anti_trainset'])
accuracy.rmse(predictions_raw_anti_trainset)
predictions_raw_testset = algo_raw.test(raw_sets['testset'])
accuracy.rmse(predictions_raw_testset)

RMSE: 54.8093
RMSE: 53.1981


53.198100848311334

In [24]:
file = open('/home/Virality_Predictor/models/CF_EN_PT_raw.pkl',"wb")
pickle.dump(algo_raw,file)
file.close()

In [15]:
# Precision@5 and Recall@5
precision5r, recall5r = cf_utils.precision_recall_k_fold(fold=5, at_k=5, thr=50, algorithm=algo_raw, complete_data=raw_sets['k_fold_set'])
print(precision5r, recall5r)

Precision at 5 0.02001869658119658
Recall at 5 0.015172387577647757
Precision at 5 0.01510498687664042
Recall at 5 0.01229296872650277
Precision at 5 0.012574044157242866
Recall at 5 0.010238462399205533
Precision at 5 0.018165089379600423
Recall at 5 0.013440478276834743
Precision at 5 0.015105008077544429
Recall at 5 0.014777086180489892
0.016193565014444945 0.013184276632136139


In [16]:
# Precision@10 and Recall@10
precision10r, recall10r = cf_utils.precision_recall_k_fold(fold=5, at_k=10, thr=50, algorithm=algo_raw, complete_data=raw_sets['k_fold_set'])
print(precision10r, recall10r)

Precision at 10 0.01349078957425667
Recall at 10 0.009868673773168158
Precision at 10 0.018739294278862624
Recall at 10 0.014377952183707578
Precision at 10 0.01849846390168971
Recall at 10 0.015083314570262787
Precision at 10 0.014910592808551991
Recall at 10 0.01520290874970678
Precision at 10 0.016091954022988506
Recall at 10 0.010103168683762107
0.00817310945863495 0.006463601796060742


In [17]:
# Top 3 predictions per user
cf_utils.print_top_n_predictions_per_user(predictions_raw_testset, shared_articles_df, article_indices, 3)

User  -445337111692715325
AI Is Here to Help You Write Emails People Will Actually Read on wired
Cafeteria compra item de Pokémon Go para aumentar movimento - e funciona on b9.com
Google lança experimentos com inteligência artificial | Google Discovery on googlediscovery
  
User  1296936597887557880
Cafeteria compra item de Pokémon Go para aumentar movimento - e funciona on b9.com
It's Time to Unlearn Everything You Know About Java Exceptions | Takipi Blog on blog.takipi
My favorite people and resources to learn Android programming from on m.signalvnoise
  
User  2542290381109225938
The End Of Apps As We Know Them - Inside Intercom on blog.intercom.stfi
US Federal Judge Rules Bitcoin is Money in Coin.mx Trial - CoinDesk on coindesk
Don't Waste Your Time on Networking Events on hbr
  
User  -8763398617720485024
Probabilistic Programming on probabilistic-programming
Seniority on merowing
Criadas para casar on revistatrip.uol.com
  
User  5676219165447745192
Stackdriver Debugger now works

Criadas para casar on revistatrip.uol.com
MITSUBISHI ASX 2.0 4X4 AWD 16V GASOLINA 4P AUTOMÁTICO - WebMotors - 17115683 on webmotors.com
  
User  1202287501580555390
Seja esperto no trabalho: Melhore a comunicação na empresa com 12 robôs on br.blog.trello
The Future of Digital Banking is Now on thefinancialbrand
Former Google career coach shares a visual trick for figuring out what to do with your life on cnbc
  
User  3217014177234377440
[PDUs] Webinars Gravados realizados pelo PMI São Paulo e pela FIXE on youwilldobetter
[Tools] How to Record your Desktop Screen with YouTube on labnol
How Google is Remaking Itself as a "Machine Learning First" Company - Backchannel on backchannel
  
User  3468719753199592246
How to Create an Exponential Mindset on hbr
Quinzena tem a melhor venda diária do ano on omundoemmovimento.blogosfera.uol.com
  
User  -6316613156648676087
Is FinTech Forcing Banking to a Tipping Point? on blog.irvingwb
How to Create an Exponential Mindset on hbr
The rise of APIs 

Bots won't replace apps. Better apps will replace apps. on dangrover
[Paper] Portfolio Management White Paper on view.iil-email
[Techniques] Será que você está fazendo "brainstorm" da forma correta? - Blog Locaweb - Notícias sobre tecnologia, programação e muito mais. on blog.locaweb.com
  
User  -7393811715153527033
Santander Vies to Become First Bank to Issue Cash on Blockchain - CoinDesk on coindesk
Things you probably didn't know you could do with Chrome's Developer Console on medium.freecodecamp
Brazil's Banco Bradesco Joins R3 Consortium - CoinDesk on coindesk
  
User  -3896187702811204367
Most Interesting APIs in 2016: Cognitive Computing on programmableweb
How to use Docker to run ASP.NET Core apps on Google App Engine on cloudplatform.googleblog
The technology behind preview photos on code.facebook
  
User  -8830250090736356260
Automate Docker with the Remote API and Ruby on sitepoint
DBA BRASIL no INTEROPMIX 2016 on fabioprado
Migrating from Parse to Bluemix on developer.ibm


User  -8399605302938582500
Tasting Dagger 2 on Android on fernandocejas
The Broken Window Theory on blog.codinghorror
How a Japanese cucumber farmer is using deep learning and TensorFlow | Google Cloud Big Data and Machine Learning Blog | Google Cloud Platform on cloud.google
  
User  7056830237116201003
Vídeos motivacionais: os 21 vídeos mais incríveis. on saiadolugar.com
The brilliant mechanics of Pokémon Go on techcrunch
  
User  -6067316262393890508
Hero unveils a new home gadget to help you track and dispense pills on techcrunch
The Digital Transformation Playbook: Rethink Your Business for the Digital Age (Columbia Business School Publishing) eBook Kindle on amazon.com
thinkseg traz o seguro "pague conforme você usa" para taxistas on revistaapolice.com
  
User  6525511498345160052
Forget The Internet Of Things, There Is A Digital Revolution Taking Place In Our Shopping Malls on forbes
Flipping the office telepresence model on techcrunch
Nova pulseira tecnológica avisa se você já 

User  7410250256723888301
Digital Health Coaching on jjhws
8 Insanely Simple Productivity Hacks on inc
  
User  3852137951376367218
bliki: StranglerApplication on martinfowler
Cubr on cbarker
Benchmarks for the Top Server-Side Swift Frameworks vs. Node.js on medium
  
User  8341545449730058951
Slice Labs raises $3.9M to insure on-demand workers when they're on the job on techcrunch
Domain Driven Design e Microservices on infoq
Who Owns Digital Transformation? According To A New Survey, It's Not The CIO on forbes
  
User  6967147874415599925
The Business Case for Progressive Web Apps on cloudfour
How to Get a Job at Google on nytimes
Adobe Solution Partner Program News on m.info.adobesystems
  
User  6947583688031316012
Progressive Web App - first introduced on Google I/O | airberlin on flights.airberlin
What to Inspect When You are Inspecting! on jerrygamblin
The Best Linux Distros of 2016 on linux
  
User  -5444946123447491034
Spanner, the Google Database That Mastered Time, Is Now Op

Um bilhão de arquivos mostram quem vence a disputa tabs vs. espaços entre programadores on gizmodo.uol.com
  
User  1573057318153030747
Como Usar o Git e o GitHub on br.udacity
  
User  -3992639648918314515
Robô acha de almoço de 12 kg a cervejas pagas pelo cidadão on otempo.com
Natura inova e cria bot para auxiliar consumidores a escolherem presentes de natal - Startupi on startupi.com
Udacity abre código de simulador de carro autônomo para quem quiser testar on startupi.com
  
User  -8121520783058973108
Sua comunidade de Teste. on agiletesters.com
  
User  -6188513246080579917
Bancos brasileiros fecham 200 agências em 2015 - Home - iG on economia.ig.com
  
User  -315113304665694510
Custo do Erro - Cinco motivos para investir em automação de testes on attps.com
Livro: Retrospectivas Divertidas on caroli
Facebook libera botão que integra rede social a sites de e-commerce on adnews.com
  
User  8180111365399889286
Why Uber Is Killing the Bank (And What That Means for Everything Else) on

User  -5145337910218163424
Novo workaholic trabalha, pratica esportes e tem tempo para a família. Conheça on gq.globo
Ray Kurzweil: The world isn't getting worse - our information is getting better on geekwire
4 lessons we can learn from Denmark about happiness at work on businessinsider
  
User  -375620451534537810
ITA está oferecendo 10 cursos gratuitos a distância - Engenharia é: on engenhariae.com
Robert Waldinger: Do que é feita uma vida boa? Lições do mais longo estudo sobre felicidade on ted
  
User  -3707046244070418512
Conheça as 10 profissões mais felizes e infelizes do mundo on catracalivre.com
Front in BH 2016 on sympla.com
  
User  202369545176637828
Design Better Forms - uxdesign.cc - User Experience Design on uxdesign
  
User  -3989802106718825881
Effective learning through code workshops | Box Blog on box
  
User  -8823950498314351783
Build Apps Faster with Salesforce - Webinar on go.pardot
  
User  -8527027076020639137
Ranking das maiores seguradoras da Europa - 2015 |

In [18]:
# Train and test normalized data
algo_norm = SlopeOne()
algo_norm.fit(normalized_sets['trainset'])
predictions_norm_anti_trainset = algo_norm.test(normalized_sets['anti_trainset'])
accuracy.rmse(predictions_norm_anti_trainset)
predictions_norm_testset = algo_norm.test(normalized_sets['testset'])
accuracy.rmse(predictions_norm_testset)

RMSE: 0.3017
RMSE: 0.5302


0.5301646904237445

In [26]:
file = open('/home/Virality_Predictor/models/CF_EN_PT_norm.pkl',"wb")
pickle.dump(algo_norm,file)
file.close()

In [19]:
# Top 3 predictions per user
cf_utils.print_top_n_predictions_per_user(predictions_norm_testset, shared_articles_df, article_indices, 3)

User  -445337111692715325
Cafeteria compra item de Pokémon Go para aumentar movimento - e funciona on b9.com
AI Is Here to Help You Write Emails People Will Actually Read on wired
Google lança experimentos com inteligência artificial | Google Discovery on googlediscovery
  
User  1296936597887557880
Cafeteria compra item de Pokémon Go para aumentar movimento - e funciona on b9.com
It's Time to Unlearn Everything You Know About Java Exceptions | Takipi Blog on blog.takipi
My favorite people and resources to learn Android programming from on m.signalvnoise
  
User  2542290381109225938
The End Of Apps As We Know Them - Inside Intercom on blog.intercom.stfi
US Federal Judge Rules Bitcoin is Money in Coin.mx Trial - CoinDesk on coindesk
Don't Waste Your Time on Networking Events on hbr
  
User  -8763398617720485024
Probabilistic Programming on probabilistic-programming
New Sessions Added To Google I/O 2016 Schedule on androidpolice
Criadas para casar on revistatrip.uol.com
  
User  56762191

  
User  548750006199694898
Como testar exceções em Java com o JUnit - Code a Test on codeatest
GE integra inteligência artificial Alexa em uma lâmpada circular on b9.com
Carteira inteligente tem trava biométrica e controle por aplicativo - Gadgets on canaltech.com
  
User  7738156533824098084
'The Simpsons' celebrates 600 episodes with a VR couch gag on engadget
  
User  4568163094592867577
Mastering Bash and Terminal on blockloop
Apple sends out invites for an October 27th event, hints that new Macs are coming on techcrunch
Android process death - and the (big) implications for your app - Inloop on medium
  
User  3576137684812235192
Progressive Web App - first introduced on Google I/O | airberlin on flights.airberlin
WeChat's competitor to Slack has arrived on techinasia
Introducing LambCI - a serverless build system on medium
  
User  -4951114616026800382
The technology behind preview photos on code.facebook
Prisma for Android is now live for all in the Google Play Store on techcru

How to Design a Walkthrough That Users Will Read on uxmovement
Design Better Forms - uxdesign.cc - User Experience Design on uxdesign
  
User  -968371571321396196
The Continuous Delivery Maturity Model on infoq
  
User  -634318177541615985
What to look for in a code review  on capgemini.github
Drupal 8.1.0 is now available on drupal
No Brasil, '25% dos celulares ainda são 'Burrofones' on m.folha.uol.com
  
User  3581462318368446161
My favorite people and resources to learn Android programming from on m.signalvnoise
The Children's Illustrated Guide to Kubernetes on deis
  
User  -8424644554119645763
Ganhe 6 meses de acesso ao Pluralsight, maior plataforma de treinamento online on medium
Learn Hiragana: The Ultimate Guide on tofugu
  
User  -4830018755429682246
Largest IoT Event Kicks Off at Santa Clara Convention Center on readwrite
Tetra Pak Cheese and Powder Systems acquires Laude on dairyreporter
Don't document your code. Code your documentation. on dev
  
User  -6874532444478397764


User  2966898457830124144
Google anuncia Android Things, plataforma para a Internet das Coisas | Google Discovery on googlediscovery
Uma arquitetura de referência para a Internet das Coisas - Parte 1 on infoq
"5G está para IoT como 4G esteve para o Smartphone", saiba as apostas da Intel para 2017 on startupi.com
  
User  -6730258785244938562
YouTube's New Messenger Means You'll Never Have to Leave YouTube on wired
Google unveils a gigapixel 'Art Camera' that lets you view paintings down to the brushstrokes on techcrunch
Projeto Ara, o smartphone modular do Google, pode chegar ao mercado em 2017 | Google Discovery on googlediscovery
  
User  -1157793243708252831
Spanner, the Google Database That Mastered Time, Is Now Open to Everyone on wired
  
User  -8784674845716296727
Don't document your code. Code your documentation. on dev
3 noções básicas essenciais para a criação de uma suíte de automação para aplicativos web on thoughtworks
Top 5 Android Testing Frameworks with Examples on test

User  -604923632438177509
Este é o melhor jeito de entender como as mulheres sofrem machismo na área de TI on brasilpost.com
My Approach to Mobile Accessibility Testing - Ministry of Testing on ministryoftesting
  
User  -2368904890804470706
Artificial intelligence is changing SEO faster than you think on techcrunch
360 million reasons to destroy all passwords - Free Code Camp on medium.freecodecamp
  
User  -4753128705824132934
Speeding up ReSharper (and Visual Studio) on jetbrains.com/help/resharper/2016
  
User  7410250256723888301
Digital Health Coaching on jjhws
8 Insanely Simple Productivity Hacks on inc
  
User  3852137951376367218
bliki: StranglerApplication on martinfowler
Benchmarks for the Top Server-Side Swift Frameworks vs. Node.js on medium
Cubr on cbarker
  
User  8341545449730058951
Slice Labs raises $3.9M to insure on-demand workers when they're on the job on techcrunch
An independent organization just ranked Google as the best cloud, beating Amazon on businessinsider


User  2783503166832565252
Running GV sprints inside corporates - learn from my mistakes - Sprint Stories on sprintstories
  
User  -5299969637554720500
How it feels to learn Javascript in 2016 on hackernoon
Novo workaholic trabalha, pratica esportes e tem tempo para a família. Conheça on gq.globo
  
User  1671960877913596679
Elopar lança o "Digio", cartão para brigar com Nubank on pagamento
Docker - Build, Ship, and Run Any App, Anywhere on goto.docker
The Future of Digital Banking is Now on thefinancialbrand
  
User  5404470824348262768
Building more accessible technology on googleblog.blogspot
Prototypal Object-Oriented Programming using JavaScript on alistapart
Mulheres são mais criativas que homens? on meioemensagem.com
  
User  -1601406818656498127
Clippy's Back: The Future of Microsoft Is Chatbots on bloomberg
  
User  4519830131595431685
SAMSUNG OCEAN - Novas turmas de treinamento de 2017 - Embarcados on embarcados.com
What the Google I/O schedule tells us about the future of An

The Three Language Acquisition Stages on blog.thelinguist
15 Awesome Sublime Text Plugins For Web Development on tutorialzine
  
User  -8909668725653743114
Empresa britânica adota licença remunerada no período menstrual on .folha.uol.com
Amor sem fronteiras: série de fotos retrata casais gays pelo mundo on catracalivre.com
  
User  1929981821863611546
The Great Library on chokochoko.wordpress
  
User  -1915495401947962903
Infográfico: Algoritmos para Aprendizado de Máquina on infoq
Former Google career coach shares a visual trick for figuring out what to do with your life on cnbc
  
User  1473125618302310292
Investimentos em coaching e mentoring aumentam nas empresas on vocerh.uol.com
  
User  -4560916320815213506
Johnson & Johnson comprará grupo suíço por US$ 30 bi on epocanegocios.globo
  
User  -6226728612139235550
Mobile marketing statistics 2016 on smartinsights
  
User  -3671569832980816109
A minha viagem à Maternidade #tetodomundo on viagensdealline
  
User  -4139451785876182601

User  3205114392436179540
Cinco competências comportamentais para você ser um bom líder on vocesa.uol.com
  
User  7025510797071058217
Java Garbage Collection Essencial on infoq
  
User  6808447563258229576
Develop your first Facebook messenger bot in PHP on blog.adnansiddiqi
  
User  -1119397949556155765
Hype Driven Development on blog.daftcode
  
User  -8704807962619440953
Kotlin in Production - What works, Whats broken on blog.dripstat
  
User  -8854674432071487111
6 exemplos de como usar computação cognitiva | EXAME.com on exame.abril.com
  
User  -8625026691457884033
Passada longa ou curta? on ativo
  
User  7072373334511597461
Discutindo Devops na Prática on thoughtworks
  
User  8745559424774781326
Google employees are adding 'Lady' to their job titles to fight sexism on theverge
  
User  3504731561932460708
A minha viagem à Maternidade #tetodomundo on viagensdealline
  
User  4597648841216101175
Itaú é pioneiro do blockchain on baguete.com
  
User  -5611964175902830423
Ranking 

In [20]:
# Precision@5 and Recall@5
precision5, recall5 = cf_utils.precision_recall_k_fold(fold=5, at_k=5, thr=0.1, algorithm=algo_norm, complete_data=normalized_sets['k_fold_set'])
print(precision5, recall5)

Precision at 5 0.32195685670261937
Recall at 5 0.3760908502537728
Precision at 5 0.3297333333333333
Recall at 5 0.37936763900101145
Precision at 5 0.32550689375506864
Recall at 5 0.37002243792910877
Precision at 5 0.3265284423179158
Recall at 5 0.3727684041473266
Precision at 5 0.31362675125561723
Recall at 5 0.3713218815868397
0.32347045547291087 0.3739142425836119


In [21]:
# Precision@10 and Recall@10
precision10, recall10 = cf_utils.precision_recall_k_fold(fold=5, at_k=10, thr=0.1, algorithm=algo_norm, complete_data=normalized_sets['k_fold_set'])
print(precision10, recall10)

Precision at 10 0.3201904506094676
Recall at 10 0.4278895253166278
Precision at 10 0.3289135194307603
Recall at 10 0.4369509587986123
Precision at 10 0.31678548826636704
Recall at 10 0.43260941308532896
Precision at 10 0.3261018990929705
Recall at 10 0.43208459928998005
Precision at 10 0.32641622289110134
Recall at 10 0.4363369439415019
0.16184075802906667 0.2165871440432051
